In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from tensorboardX import SummaryWriter 

In [ ]:

import logging
logging.basicConfig(level=logging.ERROR) 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import seaborn as sns
import pyspark
import tensorflow as tf # NOTE: TF needs to be imported before PyTorch, otherwise we get a weird initialization error
tf.get_logger().setLevel('ERROR') # only show error messages
import torch
import fastai
import surprise
import cornac

from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.utils.general_utils import get_number_processors
from recommenders.utils.gpu_utils import get_cuda_version, get_cudnn_version
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.fastai.fastai_utils import hide_fastai_progress_bar



In [ ]:
data_sizes = ["10m", "20m"] # For augmentation purposes
data_sizes2 = ["50k","40k","30k"] # For reducing 100k dataset

algorithms = ["als", "svd", "sar", "ncf", "fastai", "bpr", "bivae", "lightgcn"]
# Main focus on LightGCN

In [ ]:
tags = pd.read_csv(f"{dataset_name}/tags.csv")
tags

In [ ]:
ratings = pd.read_csv(f"{dataset_name}/ratings.csv")
ratings

In [ ]:
movies = pd.read_csv(f"{dataset_name}/movies.csv")
genre_cols = [
    "(no genres listed)", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

In [ ]:
movies.head()

#Dispaly here

In [ ]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType
from pyspark.ml.recommendation import ALS


def prepare_training_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache()

def prepare_metrics_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache(), spark.createDataFrame(test, schema).cache()

def predict_als(model, test):
    with Timer() as t:
        preds = model.transform(test)
    return preds, t

def train_als(params, data):
    symbol = ALS(**params)
    with Timer() as t:
        model = symbol.fit(data)
    return model, t

def recommend_k_als(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        # Get the cross join of all user-item pairs and score them.
        users = train.select(DEFAULT_USER_COL).distinct()
        items = train.select(DEFAULT_ITEM_COL).distinct()
        user_item = users.crossJoin(items)
        dfs_pred = model.transform(user_item)

        # Remove seen items
        dfs_pred_exclude_train = dfs_pred.alias("pred").join(
            train.alias("train"),
            (dfs_pred[DEFAULT_USER_COL] == train[DEFAULT_USER_COL])
            & (dfs_pred[DEFAULT_ITEM_COL] == train[DEFAULT_ITEM_COL]),
            how="outer",
        )
        topk_scores = dfs_pred_exclude_train.filter(
            dfs_pred_exclude_train["train." + DEFAULT_RATING_COL].isNull()
        ).select(
            "pred." + DEFAULT_USER_COL,
            "pred." + DEFAULT_ITEM_COL,
            "pred." + DEFAULT_PREDICTION_COL,
        )
    return topk_scores, t


als_params = {
    "rank": 10,
    "maxIter": 20,
    "implicitPrefs": False,
    "alpha": 0.1,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False,
    "userCol": DEFAULT_USER_COL,
    "itemCol": DEFAULT_ITEM_COL,
    "ratingCol": DEFAULT_RATING_COL,
}

In [ ]:
%%time

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Predicting time (s)", "RMSE", "MAE", "R2", "Explained Variance", "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:
    # Load the dataset
    df = movielens.load_pandas_df(
        size=data_size,
        header=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    )
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_stratified_split(df,
                                                ratio=0.75, 
                                                min_rating=1, 
                                                filter_by="item", 
                                                col_user=DEFAULT_USER_COL, 
                                                col_item=DEFAULT_ITEM_COL
                                                )
   
    # Loop through the algos
    for algo in algorithms:
        print(f"\nComputing {algo} algorithm on Movielens {data_size}")
        if algo == 'kgat':
            model, data, Ks, device, time_train = train_kgat()
            _, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
            print(metrics_dict_kgat)
            # Record results
            #summary = generate_summary('100k', algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            #df_results.loc[df_results.shape[0] + 1] = summary
            
        else:
            # Data prep for training set
            train = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            # Get model parameters
            model_params = params[algo]
            
            # Train the model
            model, time_train = trainer[algo](model_params, train)
            print(f"Training time: {time_train}s")
                    
            # Predict and evaluate
            train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            if "rating" in metrics[algo]:   
                # Predict for rating
                preds, time_rating = rating_predictor[algo](model, test)
                print(f"Rating prediction time: {time_rating}s")
                
                # Evaluate for rating
                ratings = rating_evaluator[algo](test, preds)
            else:
                ratings = None
                time_rating = np.nan
            
            if "ranking" in metrics[algo]:
                # Predict for ranking
                top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
                print(f"Ranking prediction time: {time_ranking}s")
                
                # Evaluate for rating
                rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
            else:
                rankings = None
                time_ranking = np.nan
                
            # Record results
            summary = generate_summary(data_size, algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            df_results.loc[df_results.shape[0] + 1] = summary
        
print("\nComputation changed")

In [ ]:
import plotly
import plotly.graph_objs as go

recall = df_result['Recall@k']
print(recall.min(), recall.max())
recall = (recall - recall.min()) / (recall.max() - recall.min())

fig1 = go.Scatter3d(x=df_result['embed_size'],
                    y=df_result['learning_rate'],
                    z=df_result['stacking_func'],
                    text = [item for item in recall.astype(str)],
                    marker=dict(color = recall,
                                reversescale=False,
                                colorscale='Blues',
                                size=7),
                    line=dict (width=0.02),
                    mode='markers')

#Make Plot.ly Layout
mylayout = go.Layout(scene=dict(xaxis=dict(title="embed_size"),
                                yaxis=dict(title="learning_rate"),
                                zaxis=dict(title="stacking_func")),)

#Plot and save html
plotly.offline.plot({"data": [fig1],
                     "layout": mylayout},
                     auto_open=True)
# reference: https://medium.com/@prasadostwal/multi-dimension-plots-in-python-from-2d-to-6d-9a2bf7b8cc74